PART TWO: TASK 1 (WEB CAM)

train_edge_model


In [ ]:
import tensorflow as tf
import numpy as np
import pathlib
import os

# --- Configuration Parameters ---
# Standard image size for model input (height, width)
IMG_SIZE = (160, 160)
# Batch size for datasets
BATCH_SIZE = 32
# Number of training epochs
EPOCHS = 3
# Number of classes in the dataset
NUM_CLASSES = 5
# Filename for the quantized TensorFlow Lite model
TFLITE_MODEL_NAME = 'model_quantized.tflite'
# Filename for saving the Keras model
KERAS_MODEL_PATH = 'image_classifier_keras.h5'

# --- Step 1: Data Preparation ---
print("--- 1. Preparing Data ---")

# Download and extract the flower photos dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir_base = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir_base) / 'flower_photos'

# Create training and validation datasets from directory
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Verify identified class names
class_names = train_ds.class_names
print(f"\nVerification: Found {len(class_names)} classes: {class_names}")

# Optimize dataset loading with cache, shuffle, and prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


# --- Step 2: Model Training (MobileNetV2 Transfer Learning) ---
print("\n--- 2. Training Keras Model (MobileNetV2 Transfer Learning) ---")

# Define MobileNetV2 preprocessing function (scales pixels to [-1, 1])
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Load pre-trained MobileNetV2 base model (ImageNet weights, no top layer)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze base model weights for transfer learning
base_model.trainable = False

# Global average pooling layer to reduce feature dimensions
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# Prediction layer with units equal to number of classes (outputs logits)
prediction_layer = tf.keras.layers.Dense(len(class_names))

# Construct the full Keras model
model = tf.keras.Sequential([
    tf.keras.layers.Lambda(preprocess_input, input_shape=IMG_SIZE + (3,)),
    base_model,
    global_average_layer,
    prediction_layer
])

# Compile model with Adam optimizer, sparse categorical crossentropy, and accuracy metric
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# Save the trained Keras model
model.save(KERAS_MODEL_PATH)

# Calculate and print saved Keras model size
original_size_mb = os.path.getsize(KERAS_MODEL_PATH) / (1024 * 1024)
print(f"Keras Model Saved. Original size (H5 approx): {original_size_mb:.2f} MB")


# --- Step 3: Model Conversion to TensorFlow Lite with Quantization ---
print("\n--- 3. Converting to TFLite with Quantization ---")

# Initialize TFLiteConverter from the Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Apply Dynamic Range Quantization for size reduction and speed
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the converted TFLite model to a file
tflite_model_path = TFLITE_MODEL_NAME
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# Calculate and print TFLite model size and reduction
quantized_size_mb = os.path.getsize(tflite_model_path) / (1024 * 1024)
print(f"TFLite Model Saved at: {tflite_model_path}")
print(f"Quantized size: {quantized_size_mb:.2f} MB")
print(f"Size Reduction: {100 * (1 - quantized_size_mb / original_size_mb):.2f}%")


# --- Step 4: Validate TFLite Model Accuracy (Inference) ---
print("\n--- 4. Validating TFLite Model Accuracy ---")

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize Keras Accuracy metric for TFLite validation
tflite_accuracy_metric = tf.keras.metrics.Accuracy()

def evaluate_tflite_model(interpreter, dataset, accuracy_metric):
    """Evaluates the TFLite model's accuracy on a given dataset."""
    for x, y in dataset:
        input_data = tf.cast(x, input_details[0]['dtype'])
        for i in range(input_data.shape[0]):
            # Prepare single image for inference
            input_tensor = input_data[i:i+1]
            interpreter.set_tensor(input_details[0]['index'], input_tensor)
            interpreter.invoke()
            # Get output logits
            output_data = interpreter.get_tensor(output_details[0]['index'])

            # Determine predicted class
            predicted_class = np.argmax(output_data, axis=1)

            # Update accuracy metric
            accuracy_metric.update_state(y[i].numpy(), predicted_class)

    return accuracy_metric.result().numpy()

# Run evaluation on the validation dataset
tflite_accuracy = evaluate_tflite_model(interpreter, val_ds, tflite_accuracy_metric)

print(f"\nDeployment Step Complete!")
print(f"Final Accuracy Metrics (on Validation Set):")
print(f" * Keras Model Validation Accuracy (Last Epoch): {history.history['val_accuracy'][-1]:.4f}")
print(f" * TFLite Quantized Model Accuracy: {tflite_accuracy:.4f}")

tflite_model_verification

In [ ]:
# --- TFLite Model Verification ---
# This section verifies that the TFLite model can be loaded and perform a dummy inference.

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_NAME)
interpreter.allocate_tensors()

# Get input and output tensor details (shape, data type)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Model Input Shape: {input_details[0]['shape']}")
print(f"Model Input Dtype: {input_details[0]['dtype']}")

# Create dummy input tensor (zeros) matching model's expected shape and dtype
dummy_input = np.zeros((1,) + IMG_SIZE + (3,), dtype=np.float32)

# Set dummy input, invoke interpreter, and retrieve output
interpreter.set_tensor(input_details[0]['index'], dummy_input)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print(f"\nDummy Inference Successful.")
print(f"Output shape (Logits): {output_data.shape}")
# Print predicted class index for the dummy input
print(f"Predicted Class Index (Argmax): {np.argmax(output_data)}")

single_image_inference

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Class names for interpreting model output
CLASS_NAMES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

def run_face_inference(image_path, interpreter, class_names):
    """Loads an image, preprocesses it, and runs TFLite inference to predict its class.

    Args:
        image_path (str or pathlib.Path): Path to the image file.
        interpreter (tf.lite.Interpreter): The loaded TFLite model interpreter.
        class_names (list): List of class names corresponding to model outputs.
    """

    # 1. Load and Resize Image
    img = Image.open(image_path).resize(IMG_SIZE)
    img_array = tf.keras.utils.img_to_array(img)
    input_data = np.expand_dims(img_array, axis=0)

    # 2. Preprocessing (MobileNetV2 style: scales pixels to [-1, 1], type float32)
    preprocessed_input = tf.keras.applications.mobilenet_v2.preprocess_input(input_data).astype(np.float32)

    # 3. TFLite Inference
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], preprocessed_input)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # 4. Post-processing and Result
    probabilities = tf.nn.softmax(output_data).numpy()[0]
    predicted_index = np.argmax(probabilities)
    predicted_class = class_names[predicted_index]
    confidence = probabilities[predicted_index]

    # 5. Display Result
    plt.imshow(img)
    plt.title(f"Prediction (Simulated Face Recognition):\nClass: {predicted_class} ({confidence*100:.2f}%)")
    plt.axis('off')
    plt.show()

    print(f"\n--- Simulation Result ---")
    print(f"Model Predicted: **{predicted_class}** with **{confidence*100:.2f}%** confidence.")
    print("In a real system, this confidence value would determine if the face is recognized/unlocked.")

# --- EXECUTE THE TEST ----
# Demonstrates `run_face_inference` using a sample image from the downloaded dataset.

# Get path to 'daisy' folder and find the first JPG image
daisy_dir = pathlib.Path(data_dir) / 'daisy'
try:
    test_image_path = next(daisy_dir.glob('*.jpg'))
except StopIteration:
    print("Error: No .jpg files found in the daisy directory.")
    test_image_path = None

# Run inference if a test image is found
if test_image_path:
    print(f"Successfully found test image: {test_image_path.name}")
    interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_NAME)
    interpreter.allocate_tensors()
    run_face_inference(str(test_image_path), interpreter, CLASS_NAMES)
else:
    print("Cannot run inference test without a sample image.")

In [ ]:
pip install opencv-python

webcam_test.py

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import time

# --- Configuration Parameters (Match model training) ---
TFLITE_MODEL_PATH = 'model_quantized.tflite'
IMG_SIZE = (160, 160)
CLASS_NAMES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

# --- TFLite Model Setup ---
try:
    interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
    interpreter.allocate_tensors()
except Exception as e:
    print(f"Error loading TFLite model: {e}")
    print(f"Make sure '{TFLITE_MODEL_PATH}' is in the current directory.")
    exit()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

print("TFLite Model Loaded. Starting webcam stream...")

# --- Webcam Loop (Note: This code will not work in a typical Colab environment) ---
# Intended for local Python environment with OpenCV installed.

# Initialize video capture for default webcam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # --- 1. Preprocessing Current Frame for Model Inference ---
    input_frame = cv2.resize(frame, IMG_SIZE)
    input_data = np.expand_dims(input_frame, axis=0)
    preprocessed_input = preprocess_input(input_data).astype(np.float32)

    # --- 2. TFLite Model Inference ---
    interpreter.set_tensor(input_details[0]['index'], preprocessed_input)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # --- 3. Post-processing Model Output ---
    probabilities = tf.nn.softmax(output_data).numpy()[0]
    predicted_index = np.argmax(probabilities)
    predicted_class = CLASS_NAMES[predicted_index]
    confidence = probabilities[predicted_index]

    # Calculate real-time performance metrics
    end_time = time.time()
    latency = end_time - start_time
    fps = 1.0 / latency

    # --- 4. Display Results on the Live Frame ---
    height, width, _ = frame.shape
    cv2.rectangle(frame, (50, 50), (width - 50, height - 50), (0, 255, 0), 2)

    text = f"Result: {predicted_class} ({confidence*100:.1f}%)"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    fps_text = f"FPS: {fps:.1f} | Latency: {latency*1000:.1f}ms"
    cv2.putText(frame, fps_text, (width - 250, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Edge AI Prototype (Press Q to Quit)', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and destroy windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
!python3 -m pip install ai-edge-litert

PART TWO: TASK TWO (SMART AGRICULTURAL SIMULATION SYSTEM)

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

class AgricultureSensorSimulator:
    """Generates synthetic time-series data for smart agriculture sensors over a growing period.

    This simulator aims to create realistic-looking data for sensor readings
    (soil moisture, temperature, pH) and a derived plant health index,
    which can be used for training and testing agricultural models.
    """
    def __init__(self, days=90):
        """Initializes the simulator with a specified number of days for data generation.

        Args:
            days (int): The total number of days for which to simulate data.
        """
        self.days = days
        self.timestamps = pd.to_datetime(pd.date_range(start='2024-05-01', periods=days, freq='D'))
        self.soil_moisture_base = 0.55 # Volumetric Water Content (VWC)
        self.temp_base = 25.0         # Average daily temperature (°C)
        self.ph_base = 6.5            # Soil pH

    def generate_data(self):
        """Generates synthetic time-series data for agricultural sensors.

        Returns:
            tuple: A DataFrame with sensor data and a simulated final crop yield.
        """
        # 1. Simulate Soil Moisture (cyclic + noise, clipped to realistic range)
        moisture = self.soil_moisture_base + 0.1 * np.sin(np.linspace(0, 4*np.pi, self.days))
        moisture += np.random.normal(0, 0.03, self.days)
        moisture = np.clip(moisture, 0.4, 0.7)

        # 2. Simulate Temperature (warming trend + daily fluctuations)
        temp_trend = np.linspace(-2, 3, self.days)
        temperature = self.temp_base + temp_trend + np.random.normal(0, 1.0, self.days)

        # 3. Simulate Soil pH (stable with minor fluctuations, clipped)
        ph = self.ph_base + np.random.normal(0, 0.05, self.days)
        ph = np.clip(ph, 6.0, 7.0)

        # 4. Mock Plant Health Index (PHI - growth trend + random variation, clipped)
        phi = np.linspace(0.1, 0.95, self.days)
        phi += np.random.normal(0, 0.08, self.days)
        phi = np.clip(phi, 0.0, 1.0)

        # Combine data into a DataFrame
        data = pd.DataFrame({
            'Day': range(1, self.days + 1),
            'Soil_Moisture': moisture,
            'Temperature': temperature,
            'Soil_pH': ph,
            'Plant_Health_Index': phi
        }, index=self.timestamps)

        # Calculate mock final crop yield based on PHI, moisture, and random component
        yield_score = 50 + 40 * np.mean(phi[self.days // 2:])
        yield_score += 10 * np.mean(moisture)
        final_yield = yield_score + np.random.uniform(-5, 5)

        return data, final_yield

# --- Execution Demonstration of the Simulator ---
# Create simulator instance for 120 days and generate data
sensor_data, predicted_yield = AgricultureSensorSimulator(days=120).generate_data()

print("--- Simulated IoT Sensor Data (First 5 Days) ---")
print(sensor_data.head())
print("\n--- Simulated Final Prediction Target ---")
print(f"Target Crop Yield (Tons/Hectare): {predicted_yield:.2f}")

# --- Optional: Plotting Time-Series Data for Visualization ---
plt.figure(figsize=(12, 6))
plt.plot(sensor_data.index, sensor_data['Soil_Moisture'], label='Soil Moisture (VWC)')
plt.plot(sensor_data.index, sensor_data['Temperature'] / 30, label='Temperature (Scaled)', linestyle='--')
plt.plot(sensor_data.index, sensor_data['Plant_Health_Index'], label='Plant Health Index (PHI)')
plt.title('Simulated Sensor Readings Over 120 Days')
plt.xlabel('Date')
plt.ylabel('Value (Normalized/Scaled)')
plt.legend()
plt.grid(True, alpha=0.5)
plt.show()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate

def create_hybrid_yield_model(time_steps, num_features, image_feature_vector_size):

    # --- Branch 1: Sequential Sensor Data Processing (LSTM) ---
    # Input layer for time-series sensor data (batch_size, time_steps, num_features)
    sequential_input = Input(shape=(time_steps, num_features), name='sequential_input')
    # LSTM layer to process sequence data and output a summary vector
    lstm_output = LSTM(units=64, activation='relu', name='lstm_layer')(sequential_input)

    # --- Branch 2: Image Features Input ---
    # Input layer for image-derived feature(s) (batch_size, image_feature_vector_size)
    cnn_input = Input(shape=(image_feature_vector_size,), name='cnn_feature_input')

    # --- Merging and Final Prediction (Regression Head) ---
    # Concatenate LSTM output and CNN features
    merged_features = Concatenate(name='merge_layer')([lstm_output, cnn_input])

    # Dense hidden layer for merged features
    hidden_layer = Dense(units=32, activation='relu')(merged_features)
    # Output layer for continuous crop yield prediction
    output_prediction = Dense(units=1, name='yield_output')(hidden_layer)

    # Construct the Keras Model
    model = Model(inputs=[sequential_input, cnn_input], outputs=output_prediction)

    # Compile the model with Adam optimizer, MSE loss, and MAE metric
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

# --- Execution Demonstration of Model Creation ---
# Define model parameters
TIME_STEPS = 120
NUM_FEATURES = 3
IMAGE_FEATURES = 1

# Create hybrid yield prediction model
yield_model = create_hybrid_yield_model(
    time_steps=TIME_STEPS,
    num_features=NUM_FEATURES,
    image_feature_vector_size=IMAGE_FEATURES
)

print("\n--- Mock Hybrid Model Summary (Deliverable for Task 2) ---")
# Print model architecture summary
yield_model.summary()